In [62]:
import pandas as pd
import numpy as np
import time
import sys

In [63]:
answers = pd.DataFrame.from_csv("answers-cs.csv")
answers = answers.reset_index(level=[0],drop=False)
answers = answers[answers.User.notnull()]
answers.head()

,AnsId,CreationDate,QuestionId,User
0,8,2010-08-16T20:11:04.240,2,14
1,9,2010-08-16T20:16:17.470,6,77
2,13,2010-08-16T20:31:32.583,12,80
3,16,2010-08-16T20:32:57.867,10,80
4,18,2010-08-16T20:36:48.810,10,90


In [64]:
answers.shape

(11931, 4)

In [65]:
questions = pd.DataFrame.from_csv("questions-cs.csv")
questions = questions.reset_index(level=[0],drop=False)
questions = questions[questions.QuestionId.notnull()]
questions.head()

,QuestionId,Tags
0,2,<ds.algorithms><total-ordering><sorting>
1,3,<ds.algorithms><hamiltonian-paths>
2,4,<np-hardness><set-cover><cc.complexity-theory>
3,5,<ds.algorithms>
4,6,<concurrency><process-algebra><formal-modeling>


In [66]:
answers = pd.merge(answers,questions,how='inner',on=['QuestionId'])
answers.head()

,AnsId,CreationDate,QuestionId,User,Tags
0,8,2010-08-16T20:11:04.240,2,14,<ds.algorithms><total-ordering><sorting>
1,20,2010-08-16T20:40:01.720,2,95,<ds.algorithms><total-ordering><sorting>
2,9,2010-08-16T20:16:17.470,6,77,<concurrency><process-algebra><formal-modeling>
3,13,2010-08-16T20:31:32.583,12,80,<cc.complexity-theory><lo.logic><computability>
4,48,2010-08-16T21:58:36.573,12,30,<cc.complexity-theory><lo.logic><computability>


In [67]:
#users that post more than one answers per question
print sum(answers.groupby(['User','QuestionId']).count()['AnsId']>1)
## user posts on 6 questions on avarage
print answers.groupby(['User']).count()['AnsId'].mean()

253
6.14683153014


In [68]:
# A couple utilities
get_topics = lambda string: string.strip(">").strip("<").split("><")
df_from_row = lambda row: pd.DataFrame(dict(zip(row.index,[[v] for v in row.values])))

def concat_set_tags(row,topics):
    replicated = pd.concat([df_from_row(row)]*len(topics),axis=0,ignore_index=True)
    replicated.loc[:,'TagSplit'] = pd.Series(topics)
    return replicated

replicate_for_topics = lambda row: concat_set_tags(row,get_topics(row.Tags))

rep_posts = pd.DataFrame()
for idx,row in answers.iterrows():
    rep_posts = rep_posts.append(replicate_for_topics(row),ignore_index=True)
rep_posts.head()

,AnsId,CreationDate,QuestionId,Tags,User,TagSplit
0,8,2010-08-16T20:11:04.240,2,<ds.algorithms><total-ordering><sorting>,14,ds.algorithms
1,8,2010-08-16T20:11:04.240,2,<ds.algorithms><total-ordering><sorting>,14,total-ordering
2,8,2010-08-16T20:11:04.240,2,<ds.algorithms><total-ordering><sorting>,14,sorting
3,20,2010-08-16T20:40:01.720,2,<ds.algorithms><total-ordering><sorting>,95,ds.algorithms
4,20,2010-08-16T20:40:01.720,2,<ds.algorithms><total-ordering><sorting>,95,total-ordering
5,20,2010-08-16T20:40:01.720,2,<ds.algorithms><total-ordering><sorting>,95,sorting
6,9,2010-08-16T20:16:17.470,6,<concurrency><process-algebra><formal-modeling>,77,concurrency
7,9,2010-08-16T20:16:17.470,6,<concurrency><process-algebra><formal-modeling>,77,process-algebra
8,9,2010-08-16T20:16:17.470,6,<concurrency><process-algebra><formal-modeling>,77,formal-modeling
9,13,2010-08-16T20:31:32.583,12,<cc.complexity-theory><lo.logic><computability>,80,cc.complexity-theory


In [69]:
#The vertext index
qid_tag_idx = rep_posts.groupby(['QuestionId','TagSplit']).count().reset_index(level=[0,1],drop=False)[['QuestionId','TagSplit']]
assert not qid_tag_idx.duplicated().any(),"Vertex index must be unique"
qid_tag_idx = qid_tag_idx.reset_index(level=[0],drop=False) 
qid_tag_idx.head()

,index,QuestionId,TagSplit
0,0,2,ds.algorithms
1,1,2,sorting
2,2,2,total-ordering
3,3,3,ds.algorithms
4,4,3,hamiltonian-paths


In [70]:
ans_short =  pd.merge(rep_posts,qid_tag_idx,how='inner',on=['QuestionId','TagSplit'])[['User','CreationDate','index']]
ans_short.head()

,User,CreationDate,index
0,14,2010-08-16T20:11:04.240,0
1,95,2010-08-16T20:40:01.720,0
2,14,2010-08-16T20:11:04.240,2
3,95,2010-08-16T20:40:01.720,2
4,14,2010-08-16T20:11:04.240,1


## Set timestamps to floats

In [71]:
ans_short.loc[:,'TimeStamp'] = ans_short.CreationDate.apply(
    lambda e: time.mktime(pd.Timestamp(e).timetuple())/(10**8))

In [77]:
## conversion preserves order (almost)
sum(~(ans_short.sort_values(by='TimeStamp').index == ans_short.sort_values(by='CreationDate').index))

0

## Drop cascade (vertex=QuestionId-Tag, Cascade=User)

In [81]:
#mapping of vertex_id and the actual vertex (Qid-tag)
pd.concat([qid_tag_idx['index'],qid_tag_idx['index']],axis=1).to_csv('cascades-cs.txt',index=False,header=False)

with open("cascades-cs.txt","a") as fh:
    fh.write("\n")
    # Groupby attribute defines the cascades
    for k,g in ans_short.groupby(['User']):
        df = g.sort_values(by='TimeStamp')
        fh.write(','.join(map(lambda tup:"%d,%.2f"%(tup[0],tup[1])
                     ,zip(df['index'],df['TimeStamp'])))+"\n")

## Get cascade trees

In [82]:
import snap

In [83]:
marginal_gains = {}
with open('net-out-edge.info') as f:
    next(f) #drop header
    for i,line in enumerate(f):
        src,dst,_,gain = line.split('/')[:4]
        marginal_gains[(int(src),int(dst))] = float(gain)

In [84]:
def get_graph_out(path):
    with open(path,'r') as f:
        flag = False
        for line in f:
            try:
                src,tgt = line.strip().split(",")
                if src!=tgt:
                    yield line
            except:
                continue

with open('net-out-snap.txt','w') as f:
    for line in get_graph_out('net-out.txt'):
        f.write(line)

In [85]:
infered_net = snap.TNGraph.New()
with open('net-out-snap.txt','r') as f:
    for line in f:
        src,tgt = line.strip().split(",")
        src,tgt = int(src),int(tgt)
        if not infered_net.IsNode(src):
            infered_net.AddNode(src)
        if not infered_net.IsNode(tgt):
            infered_net.AddNode(tgt)
        infered_net.AddEdge(src,tgt)

In [86]:
net_snap = pd.DataFrame.from_csv('net-out-snap.txt',index_col=False,header=None)
snap_pairs = {}
for k,row in net_snap.iterrows():
    snap_pairs[(row[0],row[1])] = 1

In [87]:
## Verify that output network infor is complete on marginal_gains

In [88]:
print len([t for t in snap_pairs if t not in marginal_gains])
print len(snap_pairs)
print len(marginal_gains)

0
1000
1000


## Tree dict

In [89]:
from collections import defaultdict

answers = pd.merge(rep_posts,qid_tag_idx,how='inner',on=['QuestionId','TagSplit'])
answers.loc[:,'TimeStamp'] = answers.CreationDate.apply(
    lambda e: time.mktime(pd.Timestamp(e).timetuple())/(10**8))
answers = answers[['User','index','TimeStamp','AnsId','TagSplit','QuestionId']]

#vertex_id:[list of cascades] e.g. inverted index
cascade_idx = answers.groupby(['index']).User.unique().apply(lambda li: [int(e) for e in li] )
cascade_trees = defaultdict(lambda: snap.TNGraph.New())

In [90]:
#print "edge (%d %d)"%(ni.GetId(), nb_id)
for n_it in infered_net.Nodes():
    for cascade in cascade_idx.loc[n_it.GetId()]:
        if not cascade_trees[cascade].IsNode(n_it.GetId()):
            cascade_trees[cascade].AddNode(n_it.GetId())
        max_gain = -1
        max_src = None
        for src_id in n_it.GetInEdges():
            if cascade not in cascade_idx.loc[src_id]:
                continue
            if marginal_gains[(src_id,n_it.GetId())]>max_gain :
                max_gain = marginal_gains[(src_id,n_it.GetId())]
                max_src = src_id
        if not max_src:
            continue
        if not cascade_trees[cascade].IsNode(max_src):
            cascade_trees[cascade].AddNode(max_src)
        cascade_trees[cascade].AddEdge(max_src,n_it.GetId())

In [91]:
print len(cascade_trees)
print np.unique(answers.User.values).size

629
1941


## Identify larger cascades

In [92]:
tree_sizes = [(cascade,cascade_trees[cascade].GetNodes(),cascade_trees[cascade].GetEdges()) 
              for cascade in cascade_trees]
tree_sizes = sorted(tree_sizes,key=lambda tup: tup[1],reverse=True)

In [124]:
get_tmstmp = lambda user,idx:answers[(answers.User==user)&(answers['index']==idx)]['TimeStamp'].values[0]
get_root = lambda user,tree: min([(n_it.GetId(),get_tmstmp(user,n_it.GetId())) for n_it in tree.Nodes()],
                            key=lambda tup:tup[1])[0]
#User, root_vertex_id(Qid-tag)
tree_roots = {cascade : get_root(cascade,cascade_trees[cascade])
                  for cascade in cascade_trees}

In [168]:
#snap_roots = {cascade:snap.GetTreeRootNId(cascade_trees[cascade])
#for cascade in cascade_trees}

##  Vertex. Edge. 3-length path counts

## Edges

In [150]:
edges = pd.DataFrame()
for i,cascade in enumerate(cascade_trees):
    src=None
    tgt=None
    split = zip(*[(e.GetSrcNId(),e.GetDstNId()) for e in cascade_trees[cascade].Edges()])
    if split:
        src,tgt = split
        edges = edges.append(pd.DataFrame({'src':src,'tgt':tgt}))
edges = edges.drop_duplicates()

In [155]:
source_tags = pd.merge(answers,edges,how='right',left_on=['index'],right_on=['src'])[['src','tgt','index','TagSplit']]
tgt_tags = pd.merge(answers,edges,how='right',left_on=['index'],right_on=['tgt'])[['src','tgt','index','TagSplit']]

In [162]:
tagged_edges = pd.merge(source_tags,tgt_tags,how='inner',on=['src','tgt'],suffixes=['_src','_tgt'])
tagged_edges = tagged_edges[['src','tgt','TagSplit_src','TagSplit_tgt']].drop_duplicates()

## vertices by dist to root

In [179]:
tree_depths = {}
for i,cascade in enumerate(cascade_trees):
    DistVec = snap.TIntPrV()
    snap.GetNodesAtHops(cascade_trees[cascade],
                            tree_roots[cascade],
                            DistVec,True)
    #use later for hist of depths
    tree_depths[cascade] = [item.GetVal1() for item in DistVec]

In [180]:
tree_by_hops = {}
for i,cascade in enumerate(cascade_trees):
    for d in tree_depths[cascade]:
        NodeVec = snap.TIntV()
        snap.GetNodesAtHop(cascade_trees[cascade],
                          tree_roots[cascade],d,NodeVec,True)
        tree_by_hops[(cascade,d)] = [v for v in NodeVec]

## Length-3 paths

In [187]:
tree_edges = []
dst = 2
for i,cascade in enumerate(cascade_trees):
    if dst not in tree_depths[cascade]:
        continue
    for n_it in cascade_trees[cascade].Nodes():
        HopV = snap.TIntV()
        snap.GetNodesAtHop(cascade_trees[cascade],
                           n_it.GetId(),dst,
                           HopV,True)
        if len([v for v in HopV])==0:
            continue
            
        new_edges = [(n_it.GetId(),
          cascade_trees[cascade].GetNI(v).GetInNId(0),
          v) for v in HopV]
        tree_edges = tree_edges + new_edges    

In [194]:
gpar,par,child = zip(*tree_edges)
len_3_paths = pd.DataFrame({'gpar':gpar,'par':par,'child':child})
len_3_paths.head()

,child,gpar,par
0,1982,146,588
1,1983,146,588
2,1984,146,588
3,746,146,742
4,747,146,742


In [196]:
atts = ['gpar','par','child','index','TagSplit']
gpar_tags = pd.merge(answers,len_3_paths,how='right',left_on=['index'],right_on=['gpar'])[atts]
par_tags = pd.merge(answers,len_3_paths,how='right',left_on=['index'],right_on=['par'])[atts]
child_tags = pd.merge(answers,len_3_paths,how='right',left_on=['index'],right_on=['child'])[atts]

In [200]:
path_feats = ['gpar','par','child','TagSplit_gpar','TagSplit_par','TagSplit_child']
tagged_3_paths = pd.merge(gpar_tags,par_tags,how='inner',on=['gpar','par','child'],suffixes=['_gpar','_par'])
tagged_3_paths = pd.merge(tagged_3_paths,child_tags,on=['gpar','par','child'],how='inner')
tagged_3_paths.rename(columns={'TagSplit': 'TagSplit_child'}, inplace=True)
tagged_3_paths = tagged_3_paths[path_feats].drop_duplicates()
tagged_3_paths

,gpar,par,child,TagSplit_gpar,TagSplit_par,TagSplit_child
0,78,5569,8731,np-intermediate,survey,books
3328,78,5569,8732,np-intermediate,survey,ds.data-structures
6656,78,5569,8733,np-intermediate,survey,soft-question
9984,78,5569,8734,np-intermediate,survey,survey
13312,78,9752,11719,np-intermediate,time-complexity,application-of-theory
21606,78,9752,11720,np-intermediate,time-complexity,big-picture
29900,78,9752,11721,np-intermediate,time-complexity,ds.algorithms
38194,78,9752,16308,np-intermediate,time-complexity,cc.complexity-theory
40196,78,9752,16309,np-intermediate,time-complexity,ds.algorithms
42198,78,9752,16310,np-intermediate,time-complexity,reference-request
